## Code for analysis of one patient (ID: 18992)
Requires that certain data files are available for loading. See all .csv files in code cells below.

Author: Avon Huxor

Date: May 2017

In [1]:
import pandas as pd
#import matplotlib as plt
import matplotlib.pyplot as plt
import numpy as np
%matplotlib inline

'Get idea of length of largest data file'
'this can take some time! so is switched to markdown for now'
!wc -l CHARTEVENTS.csv

In [2]:
!ls *.csv

ADMISSIONS.csv                INPUTEVENTS_CV.csv
CALLOUT.csv                   INPUTEVENTS_MV.csv
CAREGIVERS.csv                LABEVENTS.csv
CHARTEVENTS.csv               MICROBIOLOGYEVENTS.csv
CPTEVENTS.csv                 NOTEEVENTS.csv
DATETIMEEVENTS.csv            OUTPUTEVENTS.csv
DIAGNOSES_ICD.csv             PATIENTS.csv
DRGCODES.csv                  PRESCRIPTIONS.csv
D_CPT.csv                     PROCEDUREEVENTS_MV.csv
D_ICD_DIAGNOSES.csv           PROCEDURES_ICD.csv
D_ICD_PROCEDURES.csv          SERVICES.csv
D_ITEMS.csv                   TRANSFERS.csv
D_LABITEMS.csv                nba2013.csv
ICUSTAYS.csv                  patient_18992_chartevents.csv


In [3]:
# load list of item code number and associated names
items_df = pd.read_csv('D_ITEMS.csv',infer_datetime_format=True)


In [4]:
# load small part of chartevents to get column names
# to use for labelling the subset of data for the individual patient
# as these are not available in the grepped file

chartevents_df = pd.read_csv('CHARTEVENTS.csv',infer_datetime_format=True,nrows=20)
print(chartevents_df.head(3))
col_name_list = list(chartevents_df)

   ROW_ID  SUBJECT_ID  HADM_ID  ICUSTAY_ID  ITEMID            CHARTTIME  \
0     788          36   165660      241249  223834  2134-05-12 12:00:00   
1     789          36   165660      241249  223835  2134-05-12 12:00:00   
2     790          36   165660      241249  224328  2134-05-12 12:00:00   

             STORETIME   CGID   VALUE  VALUENUM VALUEUOM  WARNING  ERROR  \
0  2134-05-12 13:56:00  17525   15.00     15.00    L/min        0      0   
1  2134-05-12 13:56:00  17525  100.00    100.00      NaN        0      0   
2  2134-05-12 12:18:00  20823    0.37      0.37      NaN        0      0   

   RESULTSTATUS  STOPPED  
0           NaN      NaN  
1           NaN      NaN  
2           NaN      NaN  


### This has been done already, so new data file exists

The CHARTEVENTS datatable is so large that it was not possible to load the whole table into Pandas on my (AH) machine. Thus unix/grep was used to extract the data for one pataient by using the patient ID as the pattern. In this case, no false rown were captured. Short SUBJECT_ID number may generate some false rows, but these can be cleaned using the column of the Pnadas dataframe that is read in.

Only run again if this file is deleted, as takes long time!!
 
!date

!grep -w '18992' CHARTEVENTS.csv > patient_18992_chartevents.csv

!date

In [5]:
chartevents_df.head()
col_name_list = list(chartevents_df)

In [6]:
# read in patient 18992 data and parse dates to correct dtype
# note: this patient was chosen as he has the longest LOS for a single visit; 104 days
patient_18992_chartevents_df = pd.read_csv('patient_18992_chartevents.csv',parse_dates=[5,6])
# set column names to that from the basic csv file
patient_18992_chartevents_df.columns = col_name_list
patient_18992_chartevents_df.head()

/Users/ahuxor/anaconda/lib/python3.4/site-packages/IPython/core/interactiveshell.py:2723: DtypeWarning: Columns (13) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,ROW_ID,SUBJECT_ID,HADM_ID,ICUSTAY_ID,ITEMID,CHARTTIME,STORETIME,CGID,VALUE,VALUENUM,VALUEUOM,WARNING,ERROR,RESULTSTATUS,STOPPED
0,131158430,18992,123291,224638,39,2124-05-05 19:00:00,2124-05-05 18:41:00,18725,8.0 mm,NaN,NaN,NaN,NaN,NaN,NotStopd
1,131158431,18992,123291,224638,40,2124-05-05 19:00:00,2124-05-05 18:41:00,18725,Tracheostomy,NaN,NaN,NaN,NaN,NaN,NotStopd
2,131158432,18992,123291,224638,50,2124-05-05 19:00:00,2124-05-05 18:41:00,18725,20,20.0,NaN,NaN,NaN,NaN,NotStopd
3,131158433,18992,123291,224638,52,2124-05-05 19:00:00,2124-05-05 18:56:00,18386,102,102.0,mmHg,NaN,NaN,NaN,NotStopd
4,131158434,18992,123291,224638,60,2124-05-05 19:00:00,2124-05-05 18:41:00,18725,24.5,24.5,cmH2O,NaN,NaN,NaN,NotStopd


In [7]:
# get list of highest cadence items (parameters)
# to investigate the most observed parameters for this patient over time
pat_18992_top_items = patient_18992_chartevents_df.groupby('ITEMID').size().sort_values(ascending=False).head(20)
# display highest cadence parameters, as ITEMID, with number of occurances
pat_18992_top_items

ITEMID
646     2757
211     2572
128     2556
550     2556
742     2556
1484    2552
618     2541
1125    2525
212     2489
161     2481
5820    2094
5819    2094
5815    2093
8553    2089
8554    2089
8549    2088
159     1854
51      1730
8368    1728
52      1726
dtype: int64

In [8]:
# check labels of the high cadence parameters for this patient
# displays:
# ITEMID
# row_number, label
for item in pat_18992_top_items.index:
    print(item)
    print(items_df[items_df.ITEMID==item].LABEL)

646
1418    SpO2
Name: LABEL, dtype: object
211
475    Heart Rate
Name: LABEL, dtype: object
128
394    Code Status
Name: LABEL, dtype: object
550
195    Precautions
Name: LABEL, dtype: object
742
1505    calprevflg
Name: LABEL, dtype: object
1484
2755    Risk for Falls
Name: LABEL, dtype: object
618
263    Respiratory Rate
Name: LABEL, dtype: object
1125
118    Service Type
Name: LABEL, dtype: object
212
476    Heart Rhythm
Name: LABEL, dtype: object
161
425    Ectopy Type
Name: LABEL, dtype: object
5820
2469    SpO2 Alarm [Low]
Name: LABEL, dtype: object
5819
2468    Resp Alarm [Low]
Name: LABEL, dtype: object
5815
2464    HR Alarm [Low]
Name: LABEL, dtype: object
8553
4803    Resp Alarm [High]
Name: LABEL, dtype: object
8554
4804    SpO2 Alarm [High]
Name: LABEL, dtype: object
8549
4799    HR Alarm [High]
Name: LABEL, dtype: object
159
423    Ectopy Frequency
Name: LABEL, dtype: object
51
320    Arterial BP [Systolic]
Name: LABEL, dtype: object
8368
4637    Arterial BP [Diastolic]
N

In [9]:
items_df.dtypes
temp_items = items_df[items_df.LABEL.notnull()]
blood_pressure_items = temp_items[temp_items.LABEL.str.contains('BP')]
blood_pressure_items

,ROW_ID,ITEMID,LABEL,ABBREVIATION,DBSOURCE,LINKSTO,CATEGORY,UNITNAME,PARAM_TYPE,CONCEPTID
271,8,1449,Arterial BP(Rad),NaN,carevue,chartevents,NaN,NaN,NaN,NaN
293,30,4,ABI Ankle BP [Right],NaN,carevue,chartevents,NaN,NaN,NaN,NaN
294,31,5,ABI Brachial BP [Right],NaN,carevue,chartevents,NaN,NaN,NaN,NaN
295,32,6,ABP [Systolic],NaN,carevue,chartevents,NaN,NaN,NaN,NaN
320,57,51,Arterial BP [Systolic],NaN,carevue,chartevents,NaN,NaN,NaN,NaN
321,58,52,Arterial BP Mean,NaN,carevue,chartevents,NaN,NaN,NaN,NaN
332,69,63,BIPAP - BPM,NaN,carevue,chartevents,NaN,NaN,NaN,NaN
419,156,155,Doppler BP,NaN,carevue,chartevents,NaN,NaN,NaN,NaN
485,222,224,IABP Mean,NaN,carevue,chartevents,NaN,NaN,NaN,NaN
486,223,225,IABP setting,NaN,carevue,chartevents,NaN,NaN,NaN,NaN


In [10]:
items_df[items_df.ITEMID==52]

,ROW_ID,ITEMID,LABEL,ABBREVIATION,DBSOURCE,LINKSTO,CATEGORY,UNITNAME,PARAM_TYPE,CONCEPTID
321,58,52,Arterial BP Mean,NaN,carevue,chartevents,NaN,NaN,NaN,NaN


In [11]:
# get chart data for 52 = mean arterial blood pressure for this long-term patient, carevue
patient_18992_item52_df = patient_18992_chartevents_df[patient_18992_chartevents_df.ITEMID==52]

In [12]:
# get chart data for 456 = mean NBP (noninvasive blood pressure)for this long-term patient, carevue
patient_18992_item456_df = patient_18992_chartevents_df[patient_18992_chartevents_df.ITEMID==456]
len(patient_18992_item456_df)

987

In [13]:
# get chart data for 224322 = 
patient_18992_item224322_df = patient_18992_chartevents_df[patient_18992_chartevents_df.ITEMID==224322]
len(patient_18992_item224322_df)

0

In [16]:
# look at chart data for 220052 = mean blood pressure for this long-term patient, metavision
# there is no metavision mean blood pressure for this patient
patient_18992_item220052_df = patient_18992_chartevents_df[patient_18992_chartevents_df.ITEMID==220052]
len(patient_18992_item220052_df)

0

In [17]:
# get chart data for 220181 = non invasive mean blood pressure for this long-term patient, metavision
# equally there is no data from mtavision taken by non-invasive means
patient_18992_item220181_df = patient_18992_chartevents_df[patient_18992_chartevents_df.ITEMID==220181]
len(patient_18992_item220181_df)

0

In [18]:
bp_item_list

NameError: name 'bp_item_list' is not defined

In [ ]:
patient_18992_item220181_df

In [ ]:
# some high peaks but as '52' is carevue than no error

fig = patient_18992_item52_df.plot('CHARTTIME','VALUENUM',color='g')
fig.set_ylabel('MAP (mm/Hg)')
fig.set_xlabel('Time')
fig.legend_.remove()
plt.savefig("patient18992_MAP_carevue_plot.pdf", bbox_inches = 'tight',
    pad_inches = 0.1)

# there are gaps in plot suggesting Metavision may have been used for some parts of visit

In [ ]:
patient_18992_item52_df_sorted = patient_18992_item52_df.sort_values('CHARTTIME')

In [ ]:
# add itemid '456' 

fig = patient_18992_item456_df.plot('CHARTTIME','VALUENUM',figsize=(15, 6))
fig.plot(patient_18992_item52_df_sorted['CHARTTIME'],patient_18992_item52_df_sorted['VALUENUM'])
fig.set_ylabel('NBP (mm/Hg)')
fig.set_xlabel('Time')
fig.legend_.remove()
plt.savefig("patient18992_MAP_NBP_plot.pdf", bbox_inches = 'tight',
    pad_inches = 0.1)

# there are gaps in plot suggesting Metavision may have been used for some parts of visit

In [ ]:
# when do the excessive peaks occur (note that there is no error flag for carevue)?
# get values for plotting a zoomed in version
patient_18992_item52_df[patient_18992_item52_df.VALUENUM >200]

In [ ]:
# zoom into to this region

fig = patient_18992_item456_df.plot('CHARTTIME','VALUENUM',ylim=[0,300],xlim=['2124-05-27','2124-06-07'],figsize=(15, 6))
fig.plot(patient_18992_item52_df_sorted['CHARTTIME'],patient_18992_item52_df_sorted['VALUENUM'])
fig.set_ylabel('MAP & NBP (mm/Hg)')
fig.set_xlabel('Time')
fig.legend_.remove()
plt.savefig("patient18992_MAP_NBP_plot_zoomed.pdf", bbox_inches = 'tight',
    pad_inches = 0.1)